In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle


In [2]:
# Loading The data
data=pd.read_csv("datasets/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [4]:
### Data Preprocessing
## Dropping Irrelevant coumns

data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace =True)

In [5]:
## Encoding Catagorical Variables
gender_encoder = LabelEncoder()
data['Gender'] = gender_encoder.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [22]:
## Directly Encoding the Geography as germany =1, spain = 2 will not be good as model is just doing numerical calculations
##OHE
ohe = OneHotEncoder()
geo_encoder = ohe.fit_transform(data[['Geography']])
geo_encoder

KeyError: "None of [Index(['Geography'], dtype='object')] are in the [columns]"

In [7]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=ohe.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## save the encoders and the scalars

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(gender_encoder,file)

with open('geo_ohe.pkl','wb') as file:
    pickle.dump(ohe,file)


In [11]:
## Divide the data in independent and dependant and scale it
X = data.drop('Exited',axis=1)
y = data['Exited']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

with open('scalar.pkl','wb') as file:
    pickle.dump(scalar,file)



## Model Creation

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [13]:
model=Sequential([
    Dense(64,activation = 'relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation = 'relu'),
    Dense(1,activation='sigmoid')

])

model.summary()

/Users/ayushsinha/miniconda3/envs/genAI/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [15]:
#Set up the tensorboard
log_dir='logs/fit'
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [16]:
## Set Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=30,restore_best_weights=True)

In [17]:
##Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks  = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step - accuracy: 0.8210 - loss: 0.4227 - val_accuracy: 0.8535 - val_loss: 0.3609
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 473us/step - accuracy: 0.8554 - loss: 0.3565 - val_accuracy: 0.8490 - val_loss: 0.3497
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 471us/step - accuracy: 0.8620 - loss: 0.3463 - val_accuracy: 0.8510 - val_loss: 0.3434
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - accuracy: 0.8575 - loss: 0.3381 - val_accuracy: 0.8595 - val_loss: 0.3536
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 462us/step - accuracy: 0.8613 - loss: 0.3456 - val_accuracy: 0.8580 - val_loss: 0.3465
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step - accuracy: 0.8654 - loss: 0.3348 - val_accuracy: 0.8550 - val_loss: 0.3546
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step - accuracy: 0.8656 - loss: 0.3392 - val_accuracy: 0.8635 - val_loss: 0.3450
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - accuracy: 0.8591 - loss: 0

In [18]:
model.save('model.keras')

In [19]:
## Load Tensorboard Extension
%load_ext tensorboard

In [20]:
%tensorboard --logdir logs/fit

In [21]:
mo\\

SyntaxError: unexpected character after line continuation character (2022502443.py, line 1)